# Comprehensive Horse Racing Time‑Series Feature Builder

This notebook expects your **final merged DataFrame** and adds leakage‑safe time‑series features at the horse, jockey, trainer, and race‑context levels.  All features are computed using only prior observations relative to each race date.  Configure your column names below.

**Deliverables:**
1. Enriched DataFrame with time‑series features.
2. Saved outputs: `merged_timeseries.parquet` and `merged_timeseries.csv`.
3. Quick sanity plots and summary tables.

## 1) Configuration

Set the field names used in your merged DataFrame.  If unknown, leave `None` and the notebook will attempt a best‑effort autodetection from common names.  Two spaces after a sentence is intentional to match project style.

In [ ]:
# --- Configuration: set your column names here ---
CONFIG = {
    "date_col": None,            # e.g., "race_date"
    "race_id_col": None,         # e.g., "race_id"
    "horse_id_col": None,        # e.g., "horse_id"
    "jockey_id_col": None,       # e.g., "jockey_id"
    "trainer_id_col": None,      # e.g., "trainer_id"
    "track_col": None,           # e.g., "track_code" or "venue"
    "surface_col": None,         # e.g., "surface"
    "distance_col": None,        # numeric distance if available
    "class_col": None,           # race class/grade if available
    "finish_col": None,          # e.g., finish position
    "time_col": None,            # race time or speed figure (numeric)
    "odds_col": None,            # starting odds if present
    "target_col": None           # your modeling target (e.g., win_flag or finish_position)
}

# If your final merged DataFrame is not already in memory as `merged`, set a path here:
INPUT_PATH = None  # e.g., "/mnt/data/merged.parquet" or "/mnt/data/merged.csv"
INPUT_FORMAT = "parquet"  # "parquet" or "csv"

OUTPUT_PARQUET = "merged_timeseries.parquet"
OUTPUT_CSV = "merged_timeseries.csv"
RANDOM_STATE = 42

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

def _try_autoload(path, fmt):
    if path is None:
        return None
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Input file not found: {p}")
    if fmt.lower() == "parquet":
        return pd.read_parquet(p)
    elif fmt.lower() == "csv":
        return pd.read_csv(p)
    else:
        raise ValueError("INPUT_FORMAT must be 'parquet' or 'csv'.")

# Load if not present
if "merged" not in globals():
    merged = _try_autoload(INPUT_PATH, INPUT_FORMAT)

assert merged is not None, "Provide `merged` in memory or set INPUT_PATH/INPUT_FORMAT."

# Make a working copy
df = merged.copy()
print(f"Loaded DataFrame shape: {df.shape}")
df.head(3)

## 2) Autodetect and validate required columns

In [ ]:
COMMONS = {
    "date_col": ["race_date", "date", "raceDate", "dt", "event_date"],
    "race_id_col": ["race_id", "raceId", "race_key", "event_id"],
    "horse_id_col": ["horse_id", "horseId", "runner_id", "equine_id", "horse"],
    "jockey_id_col": ["jockey_id", "jockey", "rider_id", "rider"],
    "trainer_id_col": ["trainer_id", "trainer"],
    "track_col": ["track", "venue", "track_code", "course"],
    "surface_col": ["surface", "going"],
    "distance_col": ["distance", "dist"],
    "class_col": ["class", "grade", "race_class"],
    "finish_col": ["finish_position", "finish", "pos", "place"],
    "time_col": ["speed_figure", "race_time", "time", "sr", "beyer"],
    "odds_col": ["odds", "sp", "starting_price"],
    "target_col": ["win_flag", "won", "target", "label", "finish_position"]
}

cols_lower = {c.lower(): c for c in df.columns}

def resolve_col(key):
    if CONFIG.get(key) and CONFIG[key] in df.columns:
        return CONFIG[key]
    for cand in COMMONS.get(key, []):
        if cand in df.columns:
            return cand
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

RES = {k: resolve_col(k) for k in CONFIG.keys()}

missing = [k for k,v in RES.items() if v is None and k in ("date_col","race_id_col","horse_id_col")]
if missing:
    raise ValueError(f"Missing required column(s): {missing}.  Set CONFIG[...] at the top.")

# Cast date
df[RES["date_col"]] = pd.to_datetime(df[RES["date_col"]], errors="coerce")

# Sort by time to enforce leakage‑safety
df = df.sort_values([RES["horse_id_col"], RES["date_col"], RES["race_id_col"]]).reset_index(drop=True)

print("Resolved columns:")
for k,v in RES.items():
    print(f"  {k}: {v}")

## 3) Leakage‑safe time‑series features

In [ ]:
from pandas.api.types import is_numeric_dtype

DATE = RES["date_col"]
RID  = RES["race_id_col"]
HID  = RES["horse_id_col"]
JID  = RES["jockey_id_col"]
TID  = RES["trainer_id_col"]
TRACK= RES["track_col"]
SURF = RES["surface_col"]
DIST = RES["distance_col"]
CLASS= RES["class_col"]
FIN  = RES["finish_col"]
TIME = RES["time_col"]
ODDS = RES["odds_col"]

# Helper: safe lag within group
def add_lags(group, cols, lags=(1,3,5)):
    for c in cols:
        if c and c in group and is_numeric_dtype(group[c]):
            for L in lags:
                group[f"{c}_lag{L}"] = group[c].shift(L)
    return group

# Helper: rolling stats within group
def add_rolls(group, cols, windows=(3,5,10)):
    for c in cols:
        if c and c in group and is_numeric_dtype(group[c]):
            for w in windows:
                r = group[c].shift(1).rolling(w, min_periods=1)
                group[f"{c}_roll{w}_mean"] = r.mean()
                group[f"{c}_roll{w}_std"]  = r.std()
                group[f"{c}_roll{w}_min"]  = r.min()
                group[f"{c}_roll{w}_max"]  = r.max()
    return group

# Helper: exponentially‑weighted stats
def add_ewm(group, cols, alphas=(0.3, 0.6)):
    for c in cols:
        if c and c in group and is_numeric_dtype(group[c]):
            for a in alphas:
                e = group[c].shift(1).ewm(alpha=a, adjust=False)
                group[f"{c}_ewm{int(a*100)}"] = e.mean()
    return group

# Days since last race
df["days_since_last_race"] = df.groupby(HID)[DATE].diff().dt.days

# Career counts to date
df["starts_to_date"] = df.groupby(HID).cumcount()

# Horse form features (finish/time/odds)
num_cols_horse = [c for c in [FIN, TIME, ODDS] if c and c in df]
df = df.groupby(HID, group_keys=False).apply(add_lags, cols=num_cols_horse)
df = df.groupby(HID, group_keys=False).apply(add_rolls, cols=num_cols_horse)
df = df.groupby(HID, group_keys=False).apply(add_ewm, cols=num_cols_horse)

# Rolling win rate if target is a binary win flag
if RES["target_col"] and RES["target_col"] in df:
    tgt = RES["target_col"]
    if is_numeric_dtype(df[tgt]):
        for w in (3,5,10):
            df[f"winrate_roll{w}"] = df.groupby(HID)[tgt].shift(1).rolling(w, min_periods=1).mean()

# Jockey and trainer recent performance (win rates, avg finish/time)
def recent_perf(df, group_key, base_cols, prefix):
    out = pd.DataFrame(index=df.index)
    # appearances to date
    out[f"{prefix}_starts_to_date"] = df.groupby(group_key).cumcount()
    for c in base_cols:
        if c and c in df and is_numeric_dtype(df[c]):
            for w in (5,10,20):
                r = df.groupby(group_key)[c].shift(1).rolling(w, min_periods=1)
                out[f"{prefix}_{c}_roll{w}_mean"] = r.mean()
    if RES["target_col"] and RES["target_col"] in df and is_numeric_dtype(df[RES["target_col"]]):
        for w in (5,10,20):
            r = df.groupby(group_key)[RES["target_col"]].shift(1).rolling(w, min_periods=1)
            out[f"{prefix}_winrate_roll{w}"] = r.mean()
    return out

j_base = [FIN, TIME]
t_base = [FIN, TIME]

if JID:
    jp = recent_perf(df, JID, j_base, "jky")
    df = pd.concat([df, jp], axis=1)

if TID:
    tp = recent_perf(df, TID, t_base, "trn")
    df = pd.concat([df, tp], axis=1)

# Race‑context moving baselines: track/surface/time
if TRACK:
    for c in [TIME, FIN]:
        if c and c in df and is_numeric_dtype(df[c]):
            for w in (50, 200):
                r = df.groupby(TRACK)[c].shift(1).rolling(w, min_periods=10)
                df[f"{c}_{TRACK}_baseline_roll{w}"] = r.mean()

if SURF and TIME and TIME in df and is_numeric_dtype(df[TIME]):
    r = df.groupby(SURF)[TIME].shift(1).rolling(100, min_periods=10)
    df[f"{TIME}_{SURF}_baseline_roll100"] = r.mean()

# Calendar/time‑based features
df["race_dow"] = df[DATE].dt.dayofweek
df["race_month"] = df[DATE].dt.month
df["race_year"] = df[DATE].dt.year
df["race_weekofyear"] = df[DATE].dt.isocalendar().week.astype(int)

# Distance/class normalization if available
if DIST and DIST in df and is_numeric_dtype(df[DIST]) and TIME and TIME in df and is_numeric_dtype(df[TIME]):
    # pace‑like metric per unit distance
    df["time_per_unit_distance"] = df[TIME] / (df[DIST].replace(0, np.nan))
    # rolling normalized by track
    if TRACK:
        r = df.groupby([TRACK])[ "time_per_unit_distance" ].shift(1).rolling(100, min_periods=10)
        df["tpud_track_baseline_roll100"] = r.mean()

# Simple correlation context placeholder: correlation vs target using historical rows only (no NA drop leakage)
corr_cols = [c for c in df.columns if c not in [DATE, RID, HID, JID, TID, TRACK, SURF] and is_numeric_dtype(df[c])]
if RES["target_col"] and RES["target_col"] in df and is_numeric_dtype(df[RES['target_col']]):
    corr = df[corr_cols + [RES["target_col"]]].corr(numeric_only=True)[RES["target_col"]].drop(RES["target_col"], errors="ignore")
    corr = corr.sort_values(key=lambda s: s.abs(), ascending=False).head(30)
    corr_context = corr.to_frame("corr_with_target").reset_index().rename(columns={"index":"feature"})
else:
    corr_context = None

print("Feature engineering complete.  Shape:", df.shape)
df.head(3)

## 4) Save outputs

In [ ]:
df.to_parquet(OUTPUT_PARQUET, index=False)
df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved: {OUTPUT_PARQUET} and {OUTPUT_CSV}")

## 5) Quick summaries and sanity plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Basic stats for newly created columns
created_cols = [c for c in df.columns if c not in merged.columns]
summary = df[created_cols].describe().T if created_cols else pd.DataFrame()
summary.head(20)

In [ ]:
import matplotlib.pyplot as plt

# One example plot: distribution of days_since_last_race
if "days_since_last_race" in df.columns:
    plt.figure()
    df["days_since_last_race"].dropna().hist(bins=50)
    plt.title("Days Since Last Race  -  Distribution")
    plt.xlabel("Days")
    plt.ylabel("Count")
    plt.show()

# Example of rolling win rate stability by starts_to_date buckets (if exists)
if "winrate_roll5" in df.columns and "starts_to_date" in df.columns:
    tmp = df[["starts_to_date", "winrate_roll5"]].dropna()
    if not tmp.empty:
        tmp2 = tmp.groupby(pd.cut(tmp["starts_to_date"], bins=[0,5,10,20,50,100,500], include_lowest=True))["winrate_roll5"].mean()
        plt.figure()
        tmp2.plot(kind="bar")
        plt.title("Avg Win Rate (Roll5) by Career Starts")
        plt.xlabel("Career Starts Bucket")
        plt.ylabel("Average Win Rate")
        plt.tight_layout()
        plt.show()